#Imports

In [ ]:
!pip install sentence_transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 1.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 9.9 MB/s eta 0:00:00
  Created wheel for sentence_transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125923 sha256=c0a370c743135674161835eff0412e1675029a91cb86d031100f49731a1635f8
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence_transformers


In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from tqdm import tqdm
import pandas as pd
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from sentence_transformers import SentenceTransformer, util
from nltk.translate.bleu_score import corpus_bleu
from nltk.translate.meteor_score import single_meteor_score
import torch

#Prompting Zero Shot (Offensive Text)

##Preparing the data

In [ ]:
!gdown 1pr7Xq9jBglshOgfIMSDCLlWev3lzhVcq
!gdown 1YfQUQle9TZZ-qdWrV6Lodw_MjiiTbAR5
!gdown 1j6PIpLJTdkIupYI_bQS1BwJrXoPTQ20N

Downloading...
From: https://drive.google.com/uc?id=1pr7Xq9jBglshOgfIMSDCLlWev3lzhVcq
To: /content/train_en.txt
100% 221k/221k [00:00<00:00, 140MB/s]
Downloading...
From: https://drive.google.com/uc?id=1YfQUQle9TZZ-qdWrV6Lodw_MjiiTbAR5
To: /content/test_en.txt
100% 28.2k/28.2k [00:00<00:00, 83.4MB/s]
Downloading...
From: https://drive.google.com/uc?id=1j6PIpLJTdkIupYI_bQS1BwJrXoPTQ20N
To: /content/val_en.txt
100% 30.2k/30.2k [00:00<00:00, 81.4MB/s]


##Reading the files

In [ ]:
df_train = pd.read_csv("/content/train_en.txt", sep='\t')
df_test = pd.read_csv("/content/test_en.txt", sep='\t')
df_val = pd.read_csv("/content/val_en.txt", sep='\t')
df_train.head(2)

,Sentence,Label
0,Pussy nobody asked for your input.,1
1,"Ok, this makes no sense. This will create vigi...",1


In [ ]:
len(df_train)

3168

In [ ]:
len(df_test)

398

In [ ]:
len(df_val)

396

In [ ]:
len(df_test)/len(df_train)*100

12.563131313131311

##Counting the different labels

In [ ]:
value_counts = df_train['Label'].value_counts()

value_counts

1    1584
0    1584
Name: Label, dtype: int64

##Merging all the data into one

In [ ]:
sentences = df_train['Sentence'].tolist() + df_test['Sentence'].tolist() + df_val['Sentence'].tolist()
len(sentences)

3962

In [ ]:
labels = df_train['Label'].tolist() + df_test['Label'].tolist() + df_val['Label'].tolist()
len(labels)

3962

##Creating the T5 model

In [ ]:
model = AutoModelForSeq2SeqLM.from_pretrained('google/flan-t5-base')
tokenizer = AutoTokenizer.from_pretrained('google/flan-t5-base')

config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

##Creating the prompt

In [ ]:
prompt_template = 'Classify the text into offensive or non-offensive: '

##Evaluating the model

In [ ]:
true_labels = []
predicted_labels = []

# Assuming sentences and labels are your data
# Use tqdm to create a progress bar
for sentence, label in tqdm(zip(df_train['Sentence'].tolist(), df_train['Label'].tolist()), total=len(df_train['Sentence'].tolist()), desc="Processing"):
    prompt = f'{prompt_template}{sentence}'

    input_data = tokenizer(prompt, return_tensors='pt')
    input_ids = input_data.input_ids

    output = model.generate(input_ids)
    pred_label = tokenizer.decode(output[0], skip_special_tokens=True)

    # Convert "non-offensive" and "offensive" labels to 0 and 1
    pred_label = 0 if pred_label == "non-offensive" else 1

    # Append true and predicted labels to the lists
    true_labels.append(label)
    predicted_labels.append(pred_label)

    # Optionally print information for each iteration
    # print(f'Prompt: {prompt}\n'
    #       f'Sample: {sentence}\n'
    #       f'True label: {label}\n'
    #       f'Predicted label: {pred_label}\n')

# Calculate evaluation metrics


Processing:   0%|          | 0/3168 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1273: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
Processing:  44%|████▍     | 1402/3168 [14:13<17:55,  1.64it/s]


KeyboardInterrupt: ignored

In [ ]:
accuracy = accuracy_score(true_labels, predicted_labels)
precision = precision_score(true_labels, predicted_labels)
recall = recall_score(true_labels, predicted_labels)
f1 = f1_score(true_labels, predicted_labels)

# Print the results
print(f'Accuracy: {accuracy:.4f}')
print(f'Precision: {precision:.4f}')
print(f'Recall: {recall:.4f}')
print(f'F1-Score: {f1:.4f}')

Accuracy: 0.6876
Precision: 1.0000
Recall: 0.6876
F1-Score: 0.8149


##Evaluating the model (Test set)

In [ ]:
# Lists to store true and predicted labels
true_labels = []
predicted_labels = []

# Assuming sentences and labels are your data
# Use tqdm to create a progress bar
for sentence, label in tqdm(zip(df_test['Sentence'].tolist(), df_test['Label'].tolist()), total=len(df_test['Sentence'].tolist()), desc="Processing"):
    prompt = f'{prompt_template}{sentence}'

    input_data = tokenizer(prompt, return_tensors='pt')
    input_ids = input_data.input_ids

    output = model.generate(input_ids)
    pred_label = tokenizer.decode(output[0], skip_special_tokens=True)

    # Convert "non-offensive" and "offensive" labels to 0 and 1
    pred_label = 0 if pred_label == "non-offensive" else 1

    # Append true and predicted labels to the lists
    true_labels.append(label)
    predicted_labels.append(pred_label)

    # Optionally print information for each iteration
    # print(f'Prompt: {prompt}\n'
    #       f'Sample: {sentence}\n'
    #       f'True label: {label}\n'
    #       f'Predicted label: {pred_label}\n')

# Calculate evaluation metrics
accuracy = accuracy_score(true_labels, predicted_labels)
precision = precision_score(true_labels, predicted_labels)
recall = recall_score(true_labels, predicted_labels)
f1 = f1_score(true_labels, predicted_labels)

# Print the results
print(f'Accuracy: {accuracy:.4f}')
print(f'Precision: {precision:.4f}')
print(f'Recall: {recall:.4f}')
print(f'F1-Score: {f1:.4f}')


##Evaluating the model (Val set)

In [ ]:
# Lists to store true and predicted labels
true_labels = []
predicted_labels = []

# Assuming sentences and labels are your data
# Use tqdm to create a progress bar
for sentence, label in tqdm(zip(df_val['Sentence'].tolist(), df_val['Label'].tolist()), total=len(df_val['Sentence'].tolist()), desc="Processing"):
    prompt = f'{prompt_template}{sentence}'

    input_data = tokenizer(prompt, return_tensors='pt')
    input_ids = input_data.input_ids

    output = model.generate(input_ids)
    pred_label = tokenizer.decode(output[0], skip_special_tokens=True)

    # Convert "non-offensive" and "offensive" labels to 0 and 1
    pred_label = 0 if pred_label == "non-offensive" else 1

    # Append true and predicted labels to the lists
    true_labels.append(label)
    predicted_labels.append(pred_label)

    # Optionally print information for each iteration
    # print(f'Prompt: {prompt}\n'
    #       f'Sample: {sentence}\n'
    #       f'True label: {label}\n'
    #       f'Predicted label: {pred_label}\n')

# Calculate evaluation metrics
accuracy = accuracy_score(true_labels, predicted_labels)
precision = precision_score(true_labels, predicted_labels)
recall = recall_score(true_labels, predicted_labels)
f1 = f1_score(true_labels, predicted_labels)

# Print the results
print(f'Accuracy: {accuracy:.4f}')
print(f'Precision: {precision:.4f}')
print(f'Recall: {recall:.4f}')
print(f'F1-Score: {f1:.4f}')


#Few shot classification(Offensive text)



In [ ]:
!pip install sentence_transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 676.7 kB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 7.8 MB/s eta 0:00:00
  Created wheel for sentence_transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125923 sha256=2f35376eb1b887a1e2c377002883023f2f78fe8e46f73d7b1981b6c813968572
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence_transformers


##Embedding model

In [ ]:
embedding_model = SentenceTransformer('all-distilroberta-v1')
embeddings = embedding_model.encode(df_train['Sentence'].tolist(),
                                        batch_size=64,
                                        show_progress_bar=True)

.gitattributes:   0%|          | 0.00/737 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.3k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/653 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/333 [00:00<?, ?B/s]

train_script.py:   0%|          | 0.00/13.1k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

Batches:   0%|          | 0/50 [00:00<?, ?it/s]

In [ ]:
sentences = df_train['Sentence'].tolist()
labels = df_train['Label'].tolist()

##Going through the model

<font color="red">Just need to make it with different number of k

In [ ]:
true_labels = []
predicted_labels = []

for index, (sample, label) in tqdm(enumerate(zip(sentences, labels)), total=len(sentences), desc="Processing"):
        sentence_embeddings = embedding_model.encode(sample,
                                                     convert_to_tensor=True)
        results = util.semantic_search(sentence_embeddings, embeddings, top_k=10)

        example_indices = [result['corpus_id'] for result in results[0]]


        examples_prompt = ""
        for example_index in example_indices:
            #make sure that it is not the same sentence
            if example_index != index:
              example_text = sentences[example_index]
              example_label = 'offensive' if labels[example_index] == 1 else 'non-offensive'
              examples_prompt += f'Here is a text: {example_text}, which is: {example_label}\n'

        examples_prompt += f'Classify the following text: {sample} into offensive or non-offensive.'

        input_data = tokenizer(examples_prompt, return_tensors='pt')
        input_ids = input_data.input_ids

        output = model.generate(input_ids)
        pred_label = tokenizer.decode(output[0],skip_special_tokens=True)

        pred_label = 0 if pred_label == "non-offensive" else 1


        predicted_labels.append(pred_label)
        true_labels.append(label)

        # print(f'Prompt: {examples_prompt}\n'
        #       f'Sample: {sample}\n'
        #       f'True label: {label}\n'
        #       f'Predicted label: {pred_label}\n')

##Evaluating the model

In [ ]:
accuracy = accuracy_score(true_labels, predicted_labels)
precision = precision_score(true_labels, predicted_labels)
recall = recall_score(true_labels, predicted_labels)
f1 = f1_score(true_labels, predicted_labels)

# Print the results
print(f'Accuracy: {accuracy:.4f}')
print(f'Precision: {precision:.4f}')
print(f'Recall: {recall:.4f}')
print(f'F1-Score: {f1:.4f}')

Accuracy: 0.6700
Precision: 1.0000
Recall: 0.6700
F1-Score: 0.8024


#Few shot translation

##Downloading the data

In [ ]:
!gdown 1MuDMPbzadGkWgef66b1rSZsOnhfoWMcA
!gdown 12jfcFRM7GQrTvrFfS75TJw2mIT0QQF8P
!gdown 1_t8fRolG-tYVnDAfo4WASvL7k2UwNh6G
!gdown 1x5AzGBePqEL5CZrT-04I_BbPqsmZIixc
!gdown 1isKU6GHFzuxogVoSrSW1yr5md7M-17tD
!gdown 1DqBzFECvEjcmKouuCXo8KR70ZHDaZfey

Downloading...
From: https://drive.google.com/uc?id=1MuDMPbzadGkWgef66b1rSZsOnhfoWMcA
To: /content/test.modern.nltktok
100% 66.7k/66.7k [00:00<00:00, 88.2MB/s]
Downloading...
From: https://drive.google.com/uc?id=12jfcFRM7GQrTvrFfS75TJw2mIT0QQF8P
To: /content/test.original.nltktok
100% 70.8k/70.8k [00:00<00:00, 88.7MB/s]
Downloading...
From: https://drive.google.com/uc?id=1_t8fRolG-tYVnDAfo4WASvL7k2UwNh6G
To: /content/train.modern.nltktok
100% 919k/919k [00:00<00:00, 63.7MB/s]
Downloading...
From: https://drive.google.com/uc?id=1x5AzGBePqEL5CZrT-04I_BbPqsmZIixc
To: /content/train.original.nltktok
100% 980k/980k [00:00<00:00, 41.3MB/s]
Downloading...
From: https://drive.google.com/uc?id=1isKU6GHFzuxogVoSrSW1yr5md7M-17tD
To: /content/valid.modern.nltktok
100% 55.3k/55.3k [00:00<00:00, 89.5MB/s]
Downloading...
From: https://drive.google.com/uc?id=1DqBzFECvEjcmKouuCXo8KR70ZHDaZfey
To: /content/valid.original.nltktok
100% 58.1k/58.1k [00:00<00:00, 124MB/s]


In [ ]:
def path_to_list(file):
    f = open(file, "r")
    text = f.read()

    li = str.splitlines(text)
    return li

In [ ]:
original_li = path_to_list("/content/train.original.nltktok")
modern_li = path_to_list("/content/train.modern.nltktok")
original_val = path_to_list("/content/valid.original.nltktok")
modern_val = path_to_list("/content/valid.modern.nltktok")
original_test = path_to_list("/content/test.original.nltktok")
modern_test = path_to_list("/content/test.modern.nltktok")

##Creating the model

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [ ]:
model = AutoModelForSeq2SeqLM.from_pretrained('google/flan-t5-base')
tokenizer = AutoTokenizer.from_pretrained('google/flan-t5-base')

config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

##Running the few shot

In [ ]:
embedding_model = SentenceTransformer('all-distilroberta-v1')
embeddings = embedding_model.encode(modern_li,
                                        batch_size=64,
                                        show_progress_bar=True)

.gitattributes:   0%|          | 0.00/737 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.3k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/653 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/333 [00:00<?, ?B/s]

train_script.py:   0%|          | 0.00/13.1k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

Batches:   0%|          | 0/288 [00:00<?, ?it/s]

In [ ]:
embedding_model.to(device)
model.to(device)
print()

In [ ]:
true_labels = []
predicted_labels = []

for index, (src, trg) in tqdm(enumerate(zip(modern_li, original_li)), total=len(original_li), desc="Processing"):
        sentence_embeddings = embedding_model.encode(src,
                                                     convert_to_tensor=True)
        results = util.semantic_search(sentence_embeddings, embeddings, top_k=4)

        example_indices = [result['corpus_id'] for result in results[0]]


        examples_prompt = ""
        for example_index in example_indices:
            if example_index != index:
              example_src = modern_li[example_index]
              example_trg = original_li[example_index]
              examples_prompt += f'Here is a text: {example_src}, which is in modern English. Here is a rewrite of the text: {example_trg}, which is in Shakespearean English.\n'

        examples_prompt += f'The following text: {src} is in modern English. Rewrite it in Shakespearean English.'

        input_data = tokenizer(examples_prompt, return_tensors='pt').to(device)
        input_ids = input_data.input_ids

        output = model.generate(input_ids).to(device)
        pred_trg = tokenizer.decode(output[0],skip_special_tokens=True)



        predicted_labels.append(pred_trg)
        true_labels.append(trg)

        print(f'Prompt: {examples_prompt}\n'
              f'Source: {src}\n'
              f'Target: {trg}\n'
              f'Predicted target: {pred_trg}\n')

Processing:   0%|          | 1/18395 [00:00<3:11:26,  1.60it/s]

Prompt: Here is a text: I'm in a mood to beat you up pretty good ., which is in modern English. Here is a rewrite of the text: I have an humor to knock you indifferently well ., which is in Shakespearean English.
Here is a text: I swear I'll smack you if you hit me again ., which is in modern English. Here is a rewrite of the text: I swear I'll cuff you if you strike again ., which is in Shakespearean English.
Here is a text: I'm sorry I hit you ., which is in modern English. Here is a rewrite of the text: I am sorry I beat thee ., which is in Shakespearean English.
The following text: I have half a mind to hit you before you speak again . is in modern English. Rewrite it in Shakespearean English.
Source: I have half a mind to hit you before you speak again .
Target: I have a mind to strike thee ere thou speak'st .
Predicted target: I have half a mind to hit you before you speak again.



Processing:   0%|          | 2/18395 [00:01<2:57:31,  1.73it/s]

Prompt: Here is a text: I don't think it would be wise to let Mark Antony , whom Caesar is so fond of , outlive Caesar ., which is in modern English. Here is a rewrite of the text: I think it is not meet Mark Antony , so well beloved of Caesar , Should outlive Caesar ., which is in Shakespearean English.
Here is a text: As for us , you know we are Antony's , and he is Caesar's ., which is in modern English. Here is a rewrite of the text: For us , you know Whose he is we are , and that is Caesar's ., which is in Shakespearean English.
Here is a text: Mark Antony , take Caesar's body ., which is in modern English. Here is a rewrite of the text: Mark Antony , here , take you Caesar's body ., which is in Shakespearean English.
The following text: But if Antony is alive , healthy , friendly with Caesar , and not Caesar's prisoner , I'll shower you with gold and pearls . is in modern English. Rewrite it in Shakespearean English.
Source: But if Antony is alive , healthy , friendly with Caesar

Processing:   0%|          | 3/18395 [00:01<2:15:48,  2.26it/s]

...

Processing:   0%|          | 8/18395 [00:02<1:30:48,  3.37it/s]

Prompt: Here is a text: That blind head of yours was created to make me rich ., which is in modern English. Here is a rewrite of the text: That eyeless head of thine was first framed flesh To raise my fortunes ., which is in Shakespearean English.
Here is a text: Here is some money for you to kill me immediately ., which is in modern English. Here is a rewrite of the text: There is so much that thou wilt kill me straight ., which is in Shakespearean English.
Here is a text: Here's the money you wanted , master ., which is in modern English. Here is a rewrite of the text: Master , here's The gold you sent me for ., which is in Shakespearean English.
The following text: I'm going to make you a rich man . is in modern English. Rewrite it in Shakespearean English.
Source: I'm going to make you a rich man .
Target: Make thee a fortune from me .
Predicted target: I'm going to make you a rich man.



Processing:   0%|          | 9/18395 [00:03<1:50:40,  2.77it/s]

Prompt: Here is a text: Damn those words , "but yet ! " "But yet" is like a jailer about to bring out some horrible criminal ., which is in modern English. Here is a rewrite of the text: Fie upon "But yet." "But yet" is as a jailer to bring forth Some monstrous malefactor ., which is in Shakespearean English.
Here is a text: Whatever has already been done can't be undone now ., which is in modern English. Here is a rewrite of the text: Look what is done can not be now amended ., which is in Shakespearean English.
Here is a text: And then again , it's not so good , but only so-so ., which is in modern English. Here is a rewrite of the text: And yet it is not : it is but so-so ., which is in Shakespearean English.
The following text: But yet , madam , I don't like the sound of "but yet." It reverses all the good that came before it . is in modern English. Rewrite it in Shakespearean English.
Source: But yet , madam , I don't like the sound of "but yet." It reverses all the good that came

KeyboardInterrupt: ignored

In [ ]:
bleu_score = corpus_bleu(true_labels, predicted_labels)
print(f'BLEU Score: {bleu_score:.4f}')

BLEU Score: 0.0000


/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_